In [ ]:
import sqlite3
import pandas as pd

# Connexion à la base
db_path = "/content/drive/MyDrive/Projets/FDD/bd/fusion_ieee.db"
conn = sqlite3.connect(db_path)

print("🔹 Connexion réussie à la base fusion_ieee.db")

# --- 1. Nettoyage des doublons dans author_labs ---
author_labs = pd.read_sql_query("SELECT * FROM author_labs", conn)
before = len(author_labs)
author_labs = author_labs.drop_duplicates()
after = len(author_labs)
print(f"🧹 Doublons supprimés dans author_labs : {before - after}")

# --- 2. Suppression des liaisons vers labos inexistants ---
labs_ids = pd.read_sql_query("SELECT id FROM labs", conn)['id'].unique()
author_labs_clean = author_labs[author_labs['lab_id'].isin(labs_ids)]
removed = len(author_labs) - len(author_labs_clean)
print(f"🧱 Liaisons supprimées (labos inexistants) : {removed}")

# Sauvegarde du nettoyage
author_labs_clean.to_sql("author_labs", conn, if_exists="replace", index=False)
print(f"✅ Table author_labs nettoyée ({len(author_labs_clean)} lignes restantes)")

# --- 3. Extraction du pays à partir du lab_name ---
labs = pd.read_sql_query("SELECT * FROM labs", conn)

def extract_country(lab_name):
    try:
        last_part = lab_name.split(",")[-1].strip()
        return last_part if len(last_part) > 1 else "Inconnu"
    except:
        return "Inconnu"

labs["country"] = labs["lab_name"].apply(extract_country)

print("\n🌍 Exemple d'extraction de pays :")
print(labs[["lab_name", "country"]].head(10))

# Sauvegarde mise à jour de la table labs
labs.to_sql("labs", conn, if_exists="replace", index=False)
print(f"✅ Colonne 'country' ajoutée à la table labs ({len(labs)} enregistrements)")

conn.close()
print("\n🎯 Nettoyage de la base terminé avec succès !")

🔹 Connexion réussie à la base fusion_ieee.db
🧹 Doublons supprimés dans author_labs : 1802
🧱 Liaisons supprimées (labos inexistants) : 3443
✅ Table author_labs nettoyée (31768 lignes restantes)

🌍 Exemple d'extraction de pays :
                                            lab_name         country
0  Department of Computer Science, Virginia Tech,...             USA
1  Department of Colorectal Surgery, The Sixth Af...           China
2  Department of Radiology, The Sixth Affiliated ...           China
3  School of Electrical Computer and Telecommunic...       Australia
4               Intel Corporation, Chandler, AZ, USA             USA
5  Faculty of Information Technology, Brno Univer...  Czech Republic
6  Department of Computer and Software Engineerin...        Pakistan
7  Department of Fundamental and Applied Sciences...        Malaysia
8  Department of Computer Systems, Faculty of Inf...  Czech Republic
9  GE HealthCare Technology & Innovation Center, ...             USA
✅ Colonne 'cou

# **Restauration de article_authors dans la bd fusionnée**

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

# --- Chemins des bases ---
base_dir = "/content/drive/MyDrive/Projets/FDD/bd"
fusion_path = f"{base_dir}/fusion_ieee.db"
source_dbs = [
    f"{base_dir}/ieee_deep_learning.db",
    f"{base_dir}/ieee_nlp.db",
    f"{base_dir}/ieee_machine_learning.db"
]

# --- Connexion à la base fusionnée ---
fusion_conn = sqlite3.connect(fusion_path)

all_links = []

print("🔹 Fusion des tables 'article_authors' depuis les bases sources...\n")

for db in source_dbs:
    conn = sqlite3.connect(db)
    try:
        df = pd.read_sql_query("SELECT * FROM article_authors", conn)
        df["source_db"] = Path(db).name
        all_links.append(df)
        print(f"✅ {Path(db).name} : {len(df)} liaisons importées")
    except Exception as e:
        print(f"⚠️ {Path(db).name} : impossible de lire article_authors ({e})")
    finally:
        conn.close()

# --- Fusion des DataFrames ---
if all_links:
    merged = pd.concat(all_links, ignore_index=True)
    before = len(merged)
    merged = merged.drop_duplicates()
    after = len(merged)
    print(f"\n🧹 Doublons supprimés : {before - after}")

    # Vérification cohérence avec la base fusionnée
    articles = pd.read_sql_query("SELECT id FROM articles", fusion_conn)['id'].unique()
    authors = pd.read_sql_query("SELECT id FROM authors", fusion_conn)['id'].unique()
    merged = merged[merged['article_id'].isin(articles)]
    merged = merged[merged['author_id'].isin(authors)]
    print(f"✅ Liaisons valides : {len(merged)}")

    # Sauvegarde dans la base fusionnée
    merged.to_sql("article_authors", fusion_conn, if_exists="replace", index=False)
    print(f"📥 Table 'article_authors' restaurée ({len(merged)} lignes)")
else:
    print("⚠️ Aucune table 'article_authors' trouvée dans les bases sources.")

fusion_conn.close()
print("\n🎯 Restauration terminée avec succès !")

🔹 Fusion des tables 'article_authors' depuis les bases sources...

✅ ieee_deep_learning.db : 12633 liaisons importées
✅ ieee_nlp.db : 3378 liaisons importées
✅ ieee_machine_learning.db : 16397 liaisons importées

🧹 Doublons supprimés : 0
✅ Liaisons valides : 32408
📥 Table 'article_authors' restaurée (32408 lignes)

🎯 Restauration terminée avec succès !


In [30]:
import sqlite3
import pandas as pd

# Connexion à la base fusionnée
db_path = "/content/drive/MyDrive/Projets/FDD/bd/fusion_ieee.db"
conn = sqlite3.connect(db_path)
print("🔹 Connexion réussie à la base fusionnée\n")

# --- 1. Nombre de lignes par table ---
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("📋 Tables et nombre de lignes :")
for t in tables['name']:
    count = pd.read_sql_query(f"SELECT COUNT(*) AS total FROM {t}", conn)["total"][0]
    print(f"- {t} : {count}")

# --- 2. Vérification doublons ---
check_tables = ["articles", "authors", "labs", "article_authors", "author_labs", "keywords"]
print("\n🔍 Vérification doublons :")
for t in check_tables:
    df = pd.read_sql_query(f"SELECT * FROM {t}", conn)
    dup = df[df.duplicated()]
    print(f"- {t} : {len(dup)} doublons")

# --- 3. Vérification cohérence relations ---
print("\n🔗 Vérification cohérence relations :")
# article_authors
aa_missing_authors = pd.read_sql_query("""
SELECT COUNT(*) AS nb
FROM article_authors aa
LEFT JOIN authors a ON aa.author_id = a.id
WHERE a.id IS NULL
""", conn)["nb"][0]

aa_missing_articles = pd.read_sql_query("""
SELECT COUNT(*) AS nb
FROM article_authors aa
LEFT JOIN articles ar ON aa.article_id = ar.id
WHERE ar.id IS NULL
""", conn)["nb"][0]

# author_labs
al_missing_labs = pd.read_sql_query("""
SELECT COUNT(*) AS nb
FROM author_labs al
LEFT JOIN labs l ON al.lab_id = l.id
WHERE l.id IS NULL
""", conn)["nb"][0]

print(f"Auteurs manquants dans article_authors : {aa_missing_authors}")
print(f"Articles manquants dans article_authors : {aa_missing_articles}")
print(f"Labos manquants dans author_labs : {al_missing_labs}")

# --- 4. Vérification colonne country ---
labs = pd.read_sql_query("SELECT * FROM labs LIMIT 10", conn)
print("\n🌍 Aperçu labs et pays :")
print(labs[["lab_name","country"]])

conn.close()
print("\n✅ Check final terminé, tout semble correct !")

🔹 Connexion réussie à la base fusionnée

📋 Tables et nombre de lignes :
- articles : 10077
- sqlite_sequence : 3
- authors : 53493
- keywords : 159043
- author_labs : 31768
- labs : 14522
- article_authors : 32408

🔍 Vérification doublons :
- articles : 0 doublons
- authors : 0 doublons
- labs : 0 doublons
- article_authors : 0 doublons
- author_labs : 0 doublons
- keywords : 0 doublons

🔗 Vérification cohérence relations :
Auteurs manquants dans article_authors : 0
Articles manquants dans article_authors : 0
Labos manquants dans author_labs : 0

🌍 Aperçu labs et pays :
                                            lab_name         country
0  Department of Computer Science, Virginia Tech,...             USA
1  Department of Colorectal Surgery, The Sixth Af...           China
2  Department of Radiology, The Sixth Affiliated ...           China
3  School of Electrical Computer and Telecommunic...       Australia
4               Intel Corporation, Chandler, AZ, USA             USA
5  Facult